In [ ]:
import pandas as pd
import numpy as np
from datetime import date, timedelta
import time
import requests
from datetime import datetime
import webbrowser
import json
from matplotlib import pyplot as plt

In [ ]:
df1 = pd.read_csv("nifty_2023.csv")
df2 = pd.read_csv("nifty_2022.csv")
df3 = pd.read_csv("nifty_2021.csv")
df4 = pd.read_csv("nifty_2020.csv")
df5 = pd.read_csv("nifty_2019.csv")
df6 = pd.read_csv("nifty_2018.csv")

In [ ]:
df = pd.concat([df1,df2,df3,df4,df5,df6], axis = 0)

In [ ]:
df

In [ ]:
df.set_index("Date", inplace = True)

In [ ]:
date = pd.DataFrame(df.index)

In [ ]:
date.drop([0], axis = 0, inplace = True)

In [ ]:
date.rename(columns = {'Date':'next_date'}, inplace = True)

In [ ]:
date.reset_index(inplace = True)
date.drop(['index'], axis = 1, inplace = True)

In [ ]:
df.reset_index(inplace = True)

In [ ]:
dd = pd.concat([df,date],axis = 1)

In [ ]:
dd.dropna(inplace = True)

In [ ]:
#now i will take current date of nifty and get required day when i have to take trade

In [ ]:
dd['Date']=pd.to_datetime(dd['Date'])

In [ ]:
dd['day_name'] = dd['Date'].dt.day_name()

In [ ]:
dd

In [ ]:
dd.set_index('day_name', inplace = True)

In [ ]:
monday = pd.DataFrame(dd.loc['Monday'])
tuesday = pd.DataFrame(dd.loc['Tuesday'])
wednesday = pd.DataFrame(dd.loc['Wednesday'])
thursday = pd.DataFrame(dd.loc['Thursday'])
friday = pd.DataFrame(dd.loc['Friday'])

In [ ]:
monday['next_date'] = pd.to_datetime(monday['next_date'])
tuesday['next_date'] = pd.to_datetime(tuesday['next_date'])
wednesday['next_date'] = pd.to_datetime(wednesday['next_date'])
thursday['next_date'] = pd.to_datetime(thursday['next_date'])
friday['next_date'] = pd.to_datetime(friday['next_date'])

In [ ]:
jisko = pd.DataFrame(wednesday)

In [ ]:
jisko

In [ ]:
st_dist = 1350

for a in range(3):
    st_dist = st_dist + 50
    
    #variables
    start = ''
    end = ''
    exp = ''
    strike = ''
    dist = 1
    
    op = ''
    cl = ''
    hi = ''
    lo =''
    pre_cl = ''
    day = ''
    month = ''
    year = ''
    exp_day = ''
    exp_year = ''
    exp_month = ''

    my_df = pd.DataFrame(columns = ['now','exp_date','open','high','low','close','prev_close'])

    for i in range(len(jisko)):
        try:

            #to define strike price
            strike = round(jisko['Close'][i]/50)*50
            strike  = str(strike+st_dist)

            #to define start and end date
            if(jisko['next_date'][i].day/10>=1):
                day = str(jisko['next_date'][i].day)
            else:
                day = '0'+str(jisko['next_date'][i].day)

            if(jisko['next_date'][i].month/10>=1):
                month = str(jisko['next_date'][i].month)
            else:
                month = '0'+str(jisko['next_date'][i].month)
            year = str(jisko['next_date'][i].year)

            start = day+'-'+ month +'-' +year
            end = start

            #to define expiry date
            exp_date = jisko['Date'][i]+timedelta(dist)
            if(exp_date.day/10>=1):
                exp_day = str(exp_date.day)
            else:
                exp_day = '0'+str(exp_date.day)
            exp_month = exp_date.strftime("%B")[0:3]
            exp_year = str(exp_date.year)
            exp = exp_day + '-'+exp_month + '-'+exp_year

            #finally making url
            url =  f'https://www.nseindia.com/api/historical/foCPV?from={start}&to={end}&instrumentType=OPTIDX&symbol=NIFTY&year={exp_year}&expiryDate={exp}&optionType=CE&strikePrice={strike}'

            #now open in new tab to url
            webbrowser.open_new_tab(url)

            #wait for some time to open and get fucking shit
            time.sleep(1)


            headers = {
                'User-Agent': 'Mozilla/5.0',
                'Accept-Language': 'en-US,en;q=0.9',
                'Accept-Encoding': 'gzip, deflate, br',
                'Referer': 'https://www.nseindia.com',
                'X-Requested-With': 'XMLHttpRequest'
            }

            #now request
            response = requests.get(url, headers = headers)


            #now convert into simple dict
            json_string = response.content.decode('utf-8')
            data = json.loads(json_string)

            #now get required prices from dict

            if(len(data['data'])==0):
                op = float('Nan')
                cl = float('Nan')
                hi = float('Nan')
                lo =float('Nan')
                pre_cl = float('Nan')
            else:
                op = float(data['data'][0]['FH_OPENING_PRICE'])
                cl = float(data['data'][0]['FH_CLOSING_PRICE'])
                hi = float(data['data'][0]['FH_TRADE_HIGH_PRICE'])
                lo = float(data['data'][0]['FH_TRADE_LOW_PRICE'])
                pre_cl = float(data['data'][0]['FH_PREV_CLS'])


            #finally append that into my_df
            my_df.loc[i] = [start, exp_date,op,hi,lo,cl,pre_cl]

        except:
            continue
    my_df.dropna(inplace = True)
    my_df.to_csv("yo_"+jisko.index[0]+str(st_dist)+".csv")
    

In [ ]:
response.content

In [ ]:
exp_date


In [ ]:
my_df

In [ ]:
peice=0.2
last = 0
arr = []
arr.append(last)
for i in range(int(2/peice)):
    
    last = last+peice-0.00001
    arr.append(last)
arr.append(10)

In [ ]:
arr

In [ ]:
out = pd.cut(my_df['prev_close'], bins=arr, include_lowest=True)
ax = out.value_counts(sort=False).plot.bar(rot=0, color="b", figsize=(6,4))
# ax.set_xticklabels([c[1:-1].replace(","," to") for c in out.cat.categories])
plt.show()

In [ ]:
out.value_counts(sort=False)

In [ ]:
new_df = pd.DataFrame(my_df[my_df['prev_close']<=1])

In [ ]:
new_df

In [ ]:
no = len(new_df)

In [ ]:
tar = len(new_df[new_df['high']>=0.1+new_df['prev_close']])

In [ ]:
tar

In [ ]:
no

In [ ]:
strike = round(monday['Close'][0]/50)*50

In [ ]:
strike  = str(strike)

In [ ]:
if(monday['next_date'][0].day/10>=1):
    day = str(monday['next_date'][0].day)
else:
    day = '0'+str(monday['next_date'][0].day)
    
if(monday['next_date'][0].month/10>=1):
    month = str(monday['next_date'][0].month)
else:
    month = '0'+str(monday['next_date'][0].month)
year = str(monday['next_date'][0].year)

In [ ]:
start = day+'-'+ month +'-' +year

In [ ]:
end = start

In [ ]:
exp_date = monday['Date'][0]+timedelta(dist)

In [ ]:
if(exp_date.day/10>=1):
    exp_day = str(exp_date.day)
else:
    exp_day = '0'+str(exp_date.day)
exp_month = exp_date.strftime("%B")[0:3]
exp_year = str(exp_date.year)

In [ ]:
exp = exp_day + '-'+exp_month + '-'+exp_year

In [ ]:
exp

In [ ]:
url =  f'https://www.nseindia.com/api/historical/foCPV?from={start}&to={end}&instrumentType=OPTIDX&symbol=NIFTY&year={exp_year}&expiryDate={exp}&optionType=CE&strikePrice={strike}'

In [ ]:
url

In [ ]:
webbrowser.open_new_tab(url)

In [ ]:
headers = {
    'User-Agent': 'Mozilla/5.0',
    'Accept-Language': 'en-US,en;q=0.9',
    'Accept-Encoding': 'gzip, deflate, br',
    'Referer': 'https://www.nseindia.com',
    'X-Requested-With': 'XMLHttpRequest'
}

In [ ]:
response = requests.get(url, headers = headers)

In [ ]:
response.content

In [ ]:
json_string = response.content.decode('utf-8')
data = json.loads(json_string)

In [ ]:
data

In [ ]:
if(len(data['data'][0])==0):
    op = float('Nan')
    cl = float('Nan')
    hi = float('Nan')
    lo =float('Nan')
    pre_cl = float('Nan')
else:
    op = float(data['data'][0]['FH_OPENING_PRICE'])
    cl = float(data['data'][0]['FH_CLOSING_PRICE'])
    hi = float(data['data'][0]['FH_TRADE_HIGH_PRICE'])
    lo = float(data['data'][0]['FH_TRADE_LOW_PRICE'])
    pre_cl = float(data['data'][0]['FH_PREV_CLS'])
    
    

In [ ]:
my_df.loc[1] = [op,hi,lo,cl,pre_cl]

In [ ]:
my_df